<a href="https://colab.research.google.com/github/snbigft/api_rest/blob/main/Api_Rest_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install fastapi uvicorn psycopg2-binary nest-asyncio pyngrok



In [ ]:
import json
import psycopg2
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query
from pyngrok import ngrok
ngrok.set_auth_token("2tfZlVTSTp5jhPzAVI8iNoKRIwq_6NKUQdERg7XoqNdkJsuNA")

# Aplicar nest_asyncio para evitar conflictos en Colab
nest_asyncio.apply()

# Configuración de la base de datos
DB_HOST = "ep-snowy-dawn-a8omlpke-pooler.eastus2.azure.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_BcpnId62FYhS"

# Función para conectar a PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

# Crear la API con FastAPI
app = FastAPI()

@app.get("/customers")
def get_customer_data(query: str = Query(..., description="Valor a buscar en customer_data")):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        # Ejecutar consulta con parámetro seguro
        cursor.execute("SELECT customer_data FROM CUSTOMERS WHERE customer_data::text ILIKE %s", (f"%{query}%",))
        rows = cursor.fetchall()

        # Convertir resultados a JSON
        result = [json.loads(row[0]) for row in rows]  # Asumiendo que customer_data es JSON

        cursor.close()
        conn.close()

        return {"results": result}

    except Exception as e:
        return {"error": str(e)}

# Exponer la API con ngrok en Google Colab
def run_api():
    public_url = ngrok.connect(8000).public_url
    print(f"🔥 API en ejecución en: {public_url}/docs")
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Ejecutar la API
run_api()



INFO:     Started server process [209]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔥 API en ejecución en: https://6e61-35-203-150-106.ngrok-free.app/docs
